In [1]:
import pandas as pd
import gmplot
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import metrics


df = pd.read_csv("housing.csv")
df.head(10)

ModuleNotFoundError: No module named 'gmplot'

In [ ]:
df.isnull().sum()

In [ ]:
df['total_bedrooms'].fillna(df['total_bedrooms'].mean(),inplace = True)

In [ ]:
df.corr()

In [ ]:
corr_matrix = df.corr()

In [ ]:
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
df.describe()

We can see total bedrooms has a very low correlation with median income so we just impute the mean to fill nas

In [ ]:
df['total_bedrooms'].fillna(df['total_bedrooms'].mean,inplace = True)

In [ ]:
latitudes = df["latitude"]
longitudes = df["longitude"]

In [ ]:
gmap = gmplot.GoogleMapPlotter(37.88, -122.23, 10)
gmap.heatmap(latitudes, longitudes)

In [ ]:
gmap.draw("my_heatmap.html")

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10,6))
sns.distplot(df['median_house_value'],color='blue')
plt.show()

We can see the distribution of our target variable to right skewed and not normal

In [ ]:
plt.figure(figsize=(10,6))
sns.distplot(df['median_house_value'],color='red')
plt.show()

In [ ]:
ax = sns.distplot(df['median_house_value'], rug=True, hist=False)

In [ ]:
from scipy.stats import norm
ax = sns.distplot(df['median_house_value'], fit=norm, kde=False)

In [ ]:
ax = sns.scatterplot(x="longitude", y="latitude",hue="median_house_value",data=df)

In [ ]:
df.plot(kind='scatter', x='longitude', y='latitude', alpha=0.9, 
    s=df['population']/10, label='population', figsize=(14,10), 
    c='median_house_value', cmap=plt.get_cmap('cool'), colorbar=True)

In [ ]:
df.groupby(["ocean_proximity"])["median_house_value"].mean()

In [ ]:
temp2 = df.pivot_table(values='median_house_value',index=['ocean_proximity'])
print(temp2)

In [ ]:
temp2.plot(kind = 'bar')

we can see from the groups and data inland houses seem to have a much lover mean and medium house value suggesting the closer to the coast a block of houses are the higher the medium value and islands seem to contain outlier values, ocean proximity could possibly be a useful feature in predicting the house value

In [ ]:
len(df)

In [ ]:
df = df[df["ocean_proximity"] != "ISLAND"]

In [ ]:
len(df)

In [ ]:
df.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.2, figsize=(10,10))

here we see serval large horixonal lines in the scatter the largest of which is 500000 we will remove rows with 500000 to balalance the distribution a bit

In [ ]:
df = df[df["median_house_value"] < 499000]

In [ ]:
len(df)

In [ ]:
df.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.2, figsize=(10,10))

Since there is opnly 5 island districts and they are large outlier value we will remove islands from the dataset

In [ ]:
df['rooms_per_household'] = df['total_rooms']/df['households']
df['bedrooms_per_room'] = df['total_bedrooms']/df['total_rooms']

# checkout the correlations again
corr_matrix = df.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(10,6))
sns.distplot(df['median_house_value'],color='red')
plt.show()

removed the major outlier from before

In [ ]:
#x=df.iloc[:,0:7].values
y = df.iloc[:,8].values
#print(df.iloc[:,8])
#df = df.drop(["median_house_value"], axis=1)

In [ ]:
le = LabelEncoder()
#le.fit(df["ocean_proximity"])
#le.classes_
df["ocean_proximity"] = le.fit_transform(df["ocean_proximity"])

In [ ]:
x = df.values

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

#Generic function for making a classification model and accessing performance:
def classification_model(model, data, predictors, outcome):
  #Fit the model:
  model.fit(data[predictors],data[outcome])
  
  #Make predictions on training set:
  predictions = model.predict(data[predictors])
  

  #Perform k-fold cross-validation with 10 folds
  kf = KFold(data.shape[0], n_folds=10)
  error = []
  for train, test in kf:
    # Filter training data
    train_predictors = (data[predictors].iloc[train,:])
    
    # The target we're using to train the algorithm.
    train_target = data[outcome].iloc[train]
    
    # Training the algorithm using the predictors and target.
    model.fit(train_predictors, train_target)
    
    #Record error from each cross-validation run
    error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
  print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))

  #Fit the model again so that it can be refered outside the function:
  model.fit(data[predictors],data[outcome]) 

In [ ]:
#from sklearn.linear_model import LinearRegression
#outcome_var = 'median_house_value'
#predictor_var = ['ocean_proximity','rooms_per_household','median_income']
#model = LinearRegression()
#classification_model(model, df,predictor_var,outcome_var)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
linear_regressor = LinearRegression()
linear_regressor.fit(xtrain,ytrain)

In [ ]:
ypred = linear_regressor.predict(xtest)
ypred

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
predictions = linear_regressor.predict(xtest)
lin_mse = mean_squared_error(ytest,predictions)
lin_rmse = np.sqrt(lin_mse)
print('rmse value is : ',lin_rmse)

In [ ]:
lin_reg_score = linear_regressor.score(xtest,ytest)
print('r squared value is : ',lin_reg_score )

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree_regressor = DecisionTreeRegressor(random_state=0)
tree_regressor.fit(xtrain,ytrain)

In [ ]:
y_pred = tree_regressor.predict(xtest)
y_pred

In [ ]:
tree_score = tree_regressor.score(xtest,ytest)
print('r squared value is : ',tree_score )

In [ ]:
from sklearn.linear_model import LinearRegression
outcome_var = 'median_house_value'
predictor_var = ['ocean_proximity','rooms_per_household','median_income']
model = LinearRegression()
classification_model(model, df,predictor_var,outcome_var)

In [ ]:
df.head(10)

In [ ]:
from sklearn.preprocessing import StandardScaler
df_copy = df.drop(["median_house_value"], axis=1)
scaler = StandardScaler()
tree_regressor = DecisionTreeRegressor(random_state=0)
model = RandomForestClassifier(n_estimators=25, min_samples_split=10, max_depth=7, max_features=1)
#predictor_var = ['wage-increase-first-year','statutory-holidays','wage-increase-second-year',
                 #'working-hours']
#classification_model(model, df,predictor_var,outcome_var)

#print(scaler.fit(df_copy))
y = df["median_house_value"].values
x = scaler.fit_transform(df_copy)
#print(x[:])
model.fit(x, y)
model.score(x,y)
kf = KFold(x.shape[0], n_folds=10)
error = []
error1 = []
for train, test in kf:
    model.fit(x[train], y[train])
    error.append(model.score(x[test], y[test]))
    tree_regressor.fit(x[train], y[train])
    error1.append(tree_regressor.score(x[test], y[test]))
    #print(x[train])
    #tree_regressor.fit(x[train],y[train])
    #ypred = linear_regressor.predict(x[test])
    #lin_mse = mean_squared_error(ytest,predictions)
    #lin_rmse = np.sqrt(lin_mse)
    #print('rmse value is : ',lin_rmse)
    
print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))
print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error1)))

In [ ]:
#from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#labelencoder = LabelEncoder()
#print(x[:, 9])
#x[:, 8] = labelencoder.fit_transform(x[:, 8])
#onehotencoder = OneHotEncoder(categorical_features = [8])
#x = onehotencoder.fit_transform(x).toarray()
df.head(5)

In [ ]:
x=scaler.fit_transform(df[["housing_median_age","ocean_proximity","median_income","bedrooms_per_room",
                          "population","rooms_per_household"]].values)
print(x)

y=df['median_house_value'].values
print(y)

model = LinearRegression()
kf = KFold(x.shape[0], n_folds=10)
error = []

for train, test in kf:
    model.fit(x[train], y[train])
    error.append(model.score(x[test], y[test]))
    print(y[test])
    print(model.predict(x[test]))
print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))
